In [1]:
library(GWmodel)      ## GW models
library(plyr)         ## Data management
library(sp)           ## Spatial Data management
library(spdep)        ## Spatial autocorrelation
library(RColorBrewer) ## Visualization
library(classInt)     ## Class intervals
library(grid)         ## plot
library(gridExtra)    ## Multiple plot
library(ggplot2)      #  plotting
library(tidyverse)    # data 
library(SpatialML)    # Geographically weigted regression
library(h2o)
library(geojsonio)
library(pdp)       # for partial dependence plots (and ICE curves)
library(vip)       # for variable importance plots
library(matrixStats)

Loading required package: maptools

Loading required package: sp

Checking rgeos availability: TRUE
Please note that 'maptools' will be retired during 2023,
plan transition at your earliest convenience;
some functionality will be moved to 'sp'.

Loading required package: robustbase

Loading required package: Rcpp

Loading required package: spatialreg

Loading required package: spData

Loading required package: Matrix

Loading required package: sf

Linking to GEOS 3.10.2, GDAL 3.4.1, PROJ 7.2.1; sf_use_s2() is TRUE

Welcome to GWmodel version 2.2-9.


Attaching package: 'spdep'


The following objects are masked from 'package:spatialreg':

    get.ClusterOption, get.coresOption, get.mcOption,
    get.VerboseOption, get.ZeroPolicyOption, set.ClusterOption,
    set.coresOption, set.mcOption, set.VerboseOption,
    set.ZeroPolicyOption


-- Attaching core tidyverse packages ---------------------------------------------------------------- tidyverse 2.0.0 --
v dplyr     1.1.2     v readr    

In [2]:
dataFolder<- "C:\\Users\\omkarp\\Downloads\\"
df <- read.csv(paste0(dataFolder, "DATA_Sto5.csv"))

week_encoded <- model.matrix(~ factor(week) - 1, data = df)
colnames(week_encoded) <- gsub("factor\\(week\\)", "week_", colnames(week_encoded))

season_encoded <- model.matrix(~ factor(season) - 1, data = df)
colnames(season_encoded) <- gsub("factor\\(season\\)", "season_", colnames(season_encoded))

df <- cbind(df, week_encoded, season_encoded) 
df <- subset(df, select = -c(week, season))
colnames(df) <- gsub("^factor\\(week\\)", "week_", colnames(df))
colnames(df) <- gsub("^factor\\(season\\)", "season_", colnames(df))
df

,FID_grid,x,y,count_trips,TEn_ur,THin_ur,TOth_ur,Resi_lur,Comm_lur,Edu_lur,...,LUP_Other,T_Enable,T_Hinder,T_Other,week_weekday,week_weekend,season_Fall,season_Spring,season_Summer,season_Winter
,<int>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,10,671201.9,6575992,7,0,0,0,0.00001,1.00001,0.00001,...,0,0,0,0,1,0,0,0,0,1
2,10,671201.9,6575992,14,0,0,0,0.00001,1.00001,0.00001,...,0,0,0,0,0,1,0,0,0,1
3,17,674001.9,6575992,8,0,0,0,0.00001,1.00001,0.00001,...,0,0,0,0,1,0,0,0,0,1
4,18,674401.9,6575992,5,0,0,0,0.00001,0.00001,0.00001,...,62,0,0,0,1,0,0,0,0,1
5,18,674401.9,6575992,5,0,0,0,0.00001,0.00001,0.00001,...,62,0,0,0,0,1,0,0,0,1
6,19,674801.9,6575992,6,0,0,0,0.00001,0.00001,0.00001,...,0,0,0,0,1,0,0,0,0,1
7,20,675201.9,6575992,8,0,0,0,1.00001,0.00001,0.00001,...,0,0,0,0,1,0,0,0,0,1
8,21,675601.9,6575992,10,1,0,1,0.00000,0.00000,0.00000,...,0,31,0,0,0,1,1,0,0,0
9,21,675601.9,6575992,6,1,0,1,0.00000,0.00000,0.00000,...,0,31,0,0,1,0,0,0,0,1


In [3]:
set.seed(123)
split <- sample(c("train", "test"), size = nrow(df), prob = c(0.8, 0.2), replace = TRUE)
df$split <- split
df

,FID_grid,x,y,count_trips,TEn_ur,THin_ur,TOth_ur,Resi_lur,Comm_lur,Edu_lur,...,T_Enable,T_Hinder,T_Other,week_weekday,week_weekend,season_Fall,season_Spring,season_Summer,season_Winter,split
,<int>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,10,671201.9,6575992,7,0,0,0,0.00001,1.00001,0.00001,...,0,0,0,1,0,0,0,0,1,train
2,10,671201.9,6575992,14,0,0,0,0.00001,1.00001,0.00001,...,0,0,0,0,1,0,0,0,1,train
3,17,674001.9,6575992,8,0,0,0,0.00001,1.00001,0.00001,...,0,0,0,1,0,0,0,0,1,train
4,18,674401.9,6575992,5,0,0,0,0.00001,0.00001,0.00001,...,0,0,0,1,0,0,0,0,1,test
5,18,674401.9,6575992,5,0,0,0,0.00001,0.00001,0.00001,...,0,0,0,0,1,0,0,0,1,test
6,19,674801.9,6575992,6,0,0,0,0.00001,0.00001,0.00001,...,0,0,0,1,0,0,0,0,1,train
7,20,675201.9,6575992,8,0,0,0,1.00001,0.00001,0.00001,...,0,0,0,1,0,0,0,0,1,train
8,21,675601.9,6575992,10,1,0,1,0.00000,0.00000,0.00000,...,31,0,0,0,1,1,0,0,0,test
9,21,675601.9,6575992,6,1,0,1,0.00000,0.00000,0.00000,...,31,0,0,1,0,0,0,0,1,train


In [4]:
test.df<-df %>% 
  dplyr::select(FID_grid, x, y, Inc_den, AreaProx, Rd_Pedes, TEn_ur, Rd_Cyc,THin_ur, LUP_Health, Rd_Ter,
                Rd_Sec, season_Summer,week_weekend, season_Spring, count_trips) %>%
  filter(split == 'test')

valid.df<-df %>% 
  dplyr::select(FID_grid, x, y, Inc_den, AreaProx, Rd_Pedes, TEn_ur, Rd_Cyc,THin_ur, LUP_Health, Rd_Ter,
                Rd_Sec, season_Summer,week_weekend, season_Spring, count_trips) %>%
  filter(split == 'test')

train.df<-df %>% 
  dplyr::select(FID_grid, x, y, Inc_den, AreaProx, Rd_Pedes, TEn_ur, Rd_Cyc,THin_ur, LUP_Health, Rd_Ter,
                Rd_Sec, season_Summer,week_weekend, season_Spring, count_trips) %>%
  filter(split == 'train')
test.df

,FID_grid,x,y,Inc_den,AreaProx,Rd_Pedes,TEn_ur,Rd_Cyc,THin_ur,LUP_Health,Rd_Ter,Rd_Sec,season_Summer,week_weekend,season_Spring,count_trips
,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
4,18,674401.9,6575992,562.50,110894.68,2177.339,0,9894.42906,0,0,18664.849,585.2138,0,0,0,5
5,18,674401.9,6575992,562.50,110894.68,2177.339,0,9894.42906,0,0,18664.849,585.2138,0,1,0,5
8,21,675601.9,6575992,1306.25,271091.62,6493.627,1,2796.78473,0,0,16220.960,4842.5955,0,1,0,10
11,22,676001.9,6575992,1662.50,511633.52,16408.089,0,12238.17300,1,0,18068.519,3543.0685,1,1,0,6
16,43,670801.9,6576392,568.75,661394.68,7796.588,0,8157.55067,1,0,15890.888,3432.4884,0,0,0,8
20,51,674001.9,6576392,668.75,519535.54,4814.844,0,9839.50044,0,0,10209.588,7629.8148,0,1,0,7
21,53,674801.9,6576392,887.50,478571.45,15761.802,0,400.93281,0,0,14841.103,0.0000,0,0,0,11
24,54,675201.9,6576392,575.00,282618.63,6642.602,0,0.00000,0,0,9583.106,0.0000,1,1,0,6
31,55,675601.9,6576392,437.50,57158.72,2990.619,1,5439.57109,0,0,5722.658,10001.1158,0,0,0,16


In [5]:
# Scale covariates
cols_sel <- c('Inc_den', 'AreaProx', 'Rd_Pedes', 'TEn_ur', 'Rd_Cyc','THin_ur', 'LUP_Health', 'Rd_Ter',
              'Rd_Sec', 'season_Summer','week_weekend', 'season_Spring')

test.df[, cols_sel] = scale(test.df[, cols_sel])
valid.df[, cols_sel] = scale(valid.df[, cols_sel])
train.df[, cols_sel] = scale(train.df[, cols_sel])
train.df[, cols_sel]

,Inc_den,AreaProx,Rd_Pedes,TEn_ur,Rd_Cyc,THin_ur,LUP_Health,Rd_Ter,Rd_Sec,season_Summer,week_weekend,season_Spring
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,-0.5005646,-0.4922032,-1.0512819,-0.4335028,1.093721935,-0.2619083,-0.1184532,0.04069965,-0.6849439,-0.6416605,-0.9655031,-0.5205888
2,-0.5005646,-0.4922032,-1.0512819,-0.4335028,1.093721935,-0.2619083,-0.1184532,0.04069965,-0.6849439,-0.6416605,1.0352543,-0.5205888
3,-0.7309975,-0.9652339,-1.5677869,-0.4335028,0.873874366,-0.2619083,-0.1184532,-0.82233555,1.0544168,-0.6416605,-0.9655031,-0.5205888
6,-0.5208969,-1.4213096,-0.9407366,-0.4335028,0.008108647,-0.2619083,-0.1184532,0.89186358,-0.6849439,-0.6416605,-0.9655031,-0.5205888
7,-0.5028237,-1.2516327,0.3191189,-0.4335028,-0.607953406,-0.2619083,-0.1184532,0.33047917,0.5195460,-0.6416605,-0.9655031,-0.5205888
9,-0.4282719,-1.2172688,-1.0055111,2.3057321,-0.614429873,-0.2619083,-0.1184532,0.48001543,0.5273255,-0.6416605,-0.9655031,-0.5205888
10,-0.4282719,-1.2172688,-1.0055111,2.3057321,-0.614429873,-0.2619083,-0.1184532,0.48001543,0.5273255,-0.6416605,1.0352543,-0.5205888
12,-0.3989030,-0.1434613,-0.8758688,-0.4335028,-0.384214904,-0.2619083,-0.1184532,0.29176835,-0.6849439,-0.6416605,-0.9655031,-0.5205888
13,-0.3989030,-0.1434613,-0.8758688,-0.4335028,-0.384214904,-0.2619083,-0.1184532,0.29176835,-0.6849439,-0.6416605,1.0352543,-0.5205888


In [6]:
h2o.init(nthreads = -1,max_mem_size ="48g",enable_assertions = FALSE)
cols_sel1 <- c('Inc_den', 'AreaProx', 'Rd_Pedes', 'TEn_ur', 'Rd_Cyc','THin_ur', 'LUP_Health', 'Rd_Ter',
               'Rd_Sec', 'season_Summer','week_weekend', 'season_Spring','count_trips')

test.mf<-test.df[, cols_sel1] 
valid.mf<-valid.df[, cols_sel1]
train.mf<-train.df[, cols_sel1]

test.hex<-  as.h2o(test.mf)
valid.hex<-  as.h2o(valid.mf)
train.hex<-  as.h2o(train.mf)
names(train.hex)


H2O is not running yet, starting it now...

Note:  In case of errors look at the following log files:
    C:\Users\omkarp\AppData\Local\Temp\RtmpwJXPSo\file4a0c5ca05745/h2o_omkarp_started_from_r.out
    C:\Users\omkarp\AppData\Local\Temp\RtmpwJXPSo\file4a0c6b90749/h2o_omkarp_started_from_r.err


Starting H2O JVM and connecting:  Connection successful!

R is connected to the H2O cluster: 
    H2O cluster uptime:         2 seconds 44 milliseconds 
    H2O cluster timezone:       Asia/Kolkata 
    H2O data parsing timezone:  UTC 
    H2O cluster version:        3.40.0.1 
    H2O cluster version age:    4 months 
    H2O cluster name:           H2O_started_from_R_omkarp_jtw665 
    H2O cluster total nodes:    1 
    H2O cluster total memory:   42.65 GB 
    H2O cluster total cores:    12 
    H2O cluster allowed cores:  12 
    H2O cluster healthy:        TRUE 
    H2O Connection ip:          localhost 
    H2O Connection port:        54321 
    H2O Connection proxy:       NA 
    H2O Int

Warning message in h2o.clusterInfo():
"
Your H2O cluster version is (4 months) old. There may be a newer version available.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html"



  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%


[1] "Inc_den"       "AreaProx"      "Rd_Pedes"      "TEn_ur"       
 [5] "Rd_Cyc"        "THin_ur"       "LUP_Health"    "Rd_Ter"       
 [9] "Rd_Sec"        "season_Summer" "week_weekend"  "season_Spring"
[13] "count_trips"

# Local Random Forest

In [7]:
features <- as.data.frame(train.hex) %>%  dplyr::select(-count_trips)
response <- as.data.frame(train.hex) %>% pull(count_trips)

pred <- function(object, newdata)  {
  results <- as.vector(h2o.predict(object, as.h2o(newdata)))
  return(results)
  }

In [8]:
combined.df <- rbind(train.df, test.df)
combined.df

,FID_grid,x,y,Inc_den,AreaProx,Rd_Pedes,TEn_ur,Rd_Cyc,THin_ur,LUP_Health,Rd_Ter,Rd_Sec,season_Summer,week_weekend,season_Spring,count_trips
,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,10,671201.9,6575992,-0.5005646,-0.4922032,-1.0512819,-0.4335028,1.093721935,-0.2619083,-0.1184532,0.04069965,-0.6849439,-0.6416605,-0.9655031,-0.5205888,7
2,10,671201.9,6575992,-0.5005646,-0.4922032,-1.0512819,-0.4335028,1.093721935,-0.2619083,-0.1184532,0.04069965,-0.6849439,-0.6416605,1.0352543,-0.5205888,14
3,17,674001.9,6575992,-0.7309975,-0.9652339,-1.5677869,-0.4335028,0.873874366,-0.2619083,-0.1184532,-0.82233555,1.0544168,-0.6416605,-0.9655031,-0.5205888,8
6,19,674801.9,6575992,-0.5208969,-1.4213096,-0.9407366,-0.4335028,0.008108647,-0.2619083,-0.1184532,0.89186358,-0.6849439,-0.6416605,-0.9655031,-0.5205888,6
7,20,675201.9,6575992,-0.5028237,-1.2516327,0.3191189,-0.4335028,-0.607953406,-0.2619083,-0.1184532,0.33047917,0.5195460,-0.6416605,-0.9655031,-0.5205888,8
9,21,675601.9,6575992,-0.4282719,-1.2172688,-1.0055111,2.3057321,-0.614429873,-0.2619083,-0.1184532,0.48001543,0.5273255,-0.6416605,-0.9655031,-0.5205888,6
10,21,675601.9,6575992,-0.4282719,-1.2172688,-1.0055111,2.3057321,-0.614429873,-0.2619083,-0.1184532,0.48001543,0.5273255,-0.6416605,1.0352543,-0.5205888,12
12,41,670001.9,6576392,-0.3989030,-0.1434613,-0.8758688,-0.4335028,-0.384214904,-0.2619083,-0.1184532,0.29176835,-0.6849439,-0.6416605,-0.9655031,-0.5205888,5
13,41,670001.9,6576392,-0.3989030,-0.1434613,-0.8758688,-0.4335028,-0.384214904,-0.2619083,-0.1184532,0.29176835,-0.6849439,-0.6416605,1.0352543,-0.5205888,8


In [9]:
Coords<-combined.df[ ,2:3]
grf.model <- grf(count_trips ~ Inc_den+ AreaProx+ Rd_Pedes+ TEn_ur+ Rd_Cyc+THin_ur+ LUP_Health+ Rd_Ter+
                 Rd_Sec+ season_Summer+week_weekend+ season_Spring,
                 dframe=combined.df,
                 bw=101,
                 ntree= 500,
                 mtry = 2,
                 kernel="adaptive",
                 forests = TRUE,
                 coords=Coords)


Number of Observations: 2715

Number of Independent Variables: 12

Kernel: Adaptive
Neightbours: 101


--------------- Global Model Summary ---------------




Ranger result

Call:
 ranger(count_trips ~ Inc_den + AreaProx + Rd_Pedes + TEn_ur +      Rd_Cyc + THin_ur + LUP_Health + Rd_Ter + Rd_Sec + season_Summer +      week_weekend + season_Spring, data = combined.df, num.trees = 500,      mtry = 2, importance = "impurity", num.threads = NULL) 

Type:                             Regression 
Number of trees:                  500 
Sample size:                      2715 
Number of independent variables:  12 
Mtry:                             2 
Target node size:                 5 
Variable importance mode:         impurity 
Splitrule:                        variance 
OOB prediction error (MSE):       3839.993 
R squared (OOB):                  0.7543786 



Importance:




      Inc_den      AreaProx      Rd_Pedes        TEn_ur        Rd_Cyc 
    8142900.9     6358617.2     8540030.0      366089.9     2486980.2 
      THin_ur    LUP_Health        Rd_Ter        Rd_Sec season_Summer 
     239839.2      150992.6     3104821.4     2550607.4     3758097.0 
 week_weekend season_Spring 
     671318.3      385649.2 



Mean Square Error (Not OBB): 1595.304

R-squared (Not OBB) %: 89.792

AIC (Not OBB): 20048.635

AICc (Not OBB): 20048.77


--------------- Local Model Summary ---------------



Residuals OOB:




...


Residuals Predicted (Not OBB):




     Min.   1st Qu.    Median      Mean   3rd Qu.      Max. 
-274.4839   -5.5588   -0.9375    0.0859    3.2462  413.7590 



Local Variable Importance:




                    Min       Max       Mean       StD
Inc_den       249.73029 1230681.4  84378.032 164588.59
AreaProx      252.93355 1899489.1  90052.197 219165.38
Rd_Pedes      337.37398 1155796.4  67867.394 137370.96
TEn_ur         18.75284  278548.8  14710.781  31642.22
Rd_Cyc        299.69160  909570.7  59515.398 122176.54
THin_ur        18.48766  218469.1   8572.337  20265.74
LUP_Health     10.37085  110149.1   5493.321  12814.96
Rd_Ter        367.54153 1139782.9  67693.396 138649.25
Rd_Sec         75.94561 1274717.8  52901.531 139965.74
season_Summer 279.50340 4477456.8 175828.563 470831.41
week_weekend   94.05287 1036872.0  34947.739 115080.13
season_Spring  53.86055  256082.0  12966.510  29890.12



Mean squared error (OOB): 5222.743

R-squared (OOB) %: 66.581

AIC (OBB): 23268.512

AICc (OBB): 23268.647

Mean squared error Predicted (Not OBB): 865.15

R-squared Predicted (Not OBB) %: 94.464

AIC Predicted (Not OBB): 18387.281

AICc Predicted (Not OBB): 18387.416


Calculation time (in seconds): 55.4988



In [10]:
# Fetch the FID_grid and Local_R2 values from grf.model
df2 <- data.frame(FID_grid = combined.df$FID_grid, Coord = grf.model$Locations, Local_R2 = grf.model$LGofFit$LM_Rsq100, 
                  grf_Inc = grf.model$Local.Variable.Importance$Inc_den, AreaProx = grf.model$Local.Variable.Importance$AreaProx, 
                  Rd_Pedes = grf.model$Local.Variable.Importance$Rd_Pedes, TEn_ur = grf.model$Local.Variable.Importance$TEn_ur, 
                  Rd_Cyc = grf.model$Local.Variable.Importance$Rd_Cyc, THin_ur = grf.model$Local.Variable.Importance$THin_ur, 
                  LUP_Health = grf.model$Local.Variable.Importance$LUP_Health, Rd_Ter = grf.model$Local.Variable.Importance$Rd_Ter, 
                  Rd_Sec = grf.model$Local.Variable.Importance$Rd_Sec, Summer = grf.model$Local.Variable.Importance$season_Summer, 
                  weekend = grf.model$Local.Variable.Importance$week_weekend, Spring = grf.model$Local.Variable.Importance$season_Spring)
df2

,FID_grid,Coord.x,Coord.y,Local_R2,grf_Inc,AreaProx,Rd_Pedes,TEn_ur,Rd_Cyc,THin_ur,LUP_Health,Rd_Ter,Rd_Sec,Summer,weekend,Spring
,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,10,671201.9,6575992,0.2881701,1023.681,493.2883,729.9493,110.06162,377.6251,234.6535,30.04853,1566.058,442.5708,620.8049,417.9612,134.2157
2,10,671201.9,6575992,0.2509436,1066.511,514.0269,624.5236,92.47066,452.1530,256.5530,36.28877,1402.494,421.7548,493.3110,430.3670,126.2118
3,17,674001.9,6575992,0.3469470,4244.970,3456.5906,3629.6716,380.99514,4641.0099,137.0633,1574.23958,3938.527,7060.9102,3301.1458,852.0086,762.4593
6,19,674801.9,6575992,0.4354067,7292.884,8371.7398,8475.7374,1998.66372,11047.5606,1183.5886,1692.99284,11797.719,17395.5083,11938.9161,2058.3794,1554.9628
7,20,675201.9,6575992,0.4472697,6985.990,11144.8783,10058.1181,2143.36769,9282.6108,1227.8348,1192.05882,13215.493,23235.1832,16331.1444,2038.0778,2283.2700
9,21,675601.9,6575992,0.4214630,10492.049,11689.9405,12770.2988,3726.81014,7356.4965,2019.7817,1064.27970,12234.778,22172.3043,17189.4322,2836.6535,2232.9610
10,21,675601.9,6575992,0.4047344,10060.438,11326.6818,11372.1342,3070.97055,9137.0728,3141.5333,1172.90012,11135.137,22266.6198,18028.5496,2242.8530,2135.2698
12,41,670001.9,6576392,0.4649133,1006.885,661.2283,764.5071,145.32177,409.2992,162.0510,39.80872,1309.715,302.5212,1145.7213,341.7666,171.9771
13,41,670001.9,6576392,0.4363481,1221.170,621.0422,845.3320,111.89856,402.8368,179.5031,51.36511,1244.059,303.4444,1034.8979,398.1947,176.4718


In [11]:
max_values <- colMaxs(as.matrix(df2))
max_values

[1] 8.970000e+02 6.804019e+05 6.586392e+06 7.994942e-01 1.230681e+06
 [6] 1.899489e+06 1.155796e+06 2.785488e+05 9.095707e+05 2.184691e+05
[11] 1.101491e+05 1.139783e+06 1.274718e+06 4.477457e+06 1.036872e+06
[16] 2.560820e+05

In [12]:
min_values <- colMins(as.matrix(df2))
min_values

[1]  1.000000e+01  6.672019e+05  6.575992e+06 -1.953113e-02  2.497303e+02
 [6]  2.529335e+02  3.373740e+02  1.875284e+01  2.996916e+02  1.848766e+01
[11]  1.037085e+01  3.675415e+02  7.594561e+01  2.795034e+02  9.405287e+01
[16]  5.386055e+01

In [13]:
write.csv(df2,file='Sto_R2.csv')

## Local Model Summary and goodness of fit statistics

In [14]:
grf.model$LocalModelSummary

,Min,Max,Mean,StD
,<dbl>,<dbl>,<dbl>,<dbl>
Inc_den,249.73029,1230681.4,84378.032,164588.59
AreaProx,252.93355,1899489.1,90052.197,219165.38
Rd_Pedes,337.37398,1155796.4,67867.394,137370.96
TEn_ur,18.75284,278548.8,14710.781,31642.22
Rd_Cyc,299.69160,909570.7,59515.398,122176.54
THin_ur,18.48766,218469.1,8572.337,20265.74
LUP_Health,10.37085,110149.1,5493.321,12814.96
Rd_Ter,367.54153,1139782.9,67693.396,138649.25
Rd_Sec,75.94561,1274717.8,52901.531,139965.74
